## TP 1 - Algoritmos II

In [1]:
# Classe utilitária que representa um Nó individual em uma k-d Tree 

class Node:
    def __init__(self, is_leaf, considered_coordinate, position, point):
        """
        Controi um Nó para uma k-d Tree
        
        is_leaf: Booleano que indica se esse Nó é uma Folha na árvore ou se é um Nó interno 
        True ou False
        
        considered_coordinate: Qual coordenada dos pontos foi analisada para se construir esse Nó 
        0..n-1, onde n é a dimensão (qtde de coordenadas) dos pontos
        
        position: Posição ou valor do ponto na coordenada considerada
        Float
        OBS: Se 'point is not None' então position = point.positions[considered_coordinate]
        
        point: Se esse Nó não for interno (for folha), ele deve ter um 'point' como contúdo 
        """
        
        self.left = None
        self.right = None
        self.is_leaf = is_leaf 
        self.considered_coordinate = considered_coordinate
        self.position = position
        
        self.point = point

In [2]:
# Classe utilitária que representa um ponto em um espaço k-dimensional qualquer

class Point:
    
    def __init__(self, positions, classification = None):
        """
        positions: Vetor (de tamanho k) de floats que representa a posição desse ponto em um espaço k-dimensional
        (Valores em X)
        
        classification: A classificação ou rótulo real desse ponto (Valor em y)
        
        predicted_classification: A classificação ou rótulo desse ponto (Valor em y') que foi computada pelo kNN
        caso esse ponto esteja no conjunto de teste. Esse valor não é recebido pelo construtor, mas deve ser 
        inicializado como None
        """
        self.positions = positions
        self.classification = classification
        self.predicted_classification = None
    
    def __str__(self):
        point_as_str = "[" + (", ".join(str(e) for e in self.positions)) + "]"
        
        if self.classification is not None:
            point_as_str = point_as_str + "-(" + str(self.classification) + ")"
        
        if self.predicted_classification is not None:
            point_as_str = point_as_str + "-{" + str(self.predicted_classification) + "}"
        
        return point_as_str
    
    def __repr__(self):
        point_as_str = "[" + (", ".join(str(e) for e in self.positions)) + "]"
        
        if self.classification is not None:
            point_as_str = point_as_str + "-(" + str(self.classification) + ")"
        
        if self.predicted_classification is not None:
            point_as_str = point_as_str + "-{" + str(self.predicted_classification) + "}"
        
        return point_as_str

In [3]:
# Função utilitária que recebe uma lista de pontos e uma determinada coordenada desses pontos e retorna
# uma nova lista contendo os mesmos pontos ordenados com base na coordenada passada como parâmetro

# OBS: Eu pretendia usar esse método para encontrar o ponto mediano: Uma vez que a lista estivesse ordenada,
# o ponto mediano estaria trivialmente na posição len(lista) / 2. Além disso, nessa mesma lista, todos os pontos
# à esquerda seriam menores que o mediano e todos à direita seriam maiores que o mediano, logo, seria 
# fácil fazer a separação dos pontos.
# Porém, usando algoritmos com a técnica "prune-and-search", é possível encontrar o ponto mediano e
# separações dos conjuntos de pontos igualmente satisfatórias (elas não necessariamente estariam ordenadas,
# mas isso não é necessário para o nosso caso) para nosso problema de gerar a k-d-tree em tempo linear 
# (usando merge-sort, teriamos uma complexidade desnecessária de O(n log(n)))

"""def merge(left, right, coordinate):
    l_counter = r_counter = 0
    sorted_arr = []
    
    while l_counter < len(left) and r_counter < len(right):
        val_left = left[l_counter]
        val_right = right[r_counter]
        
        if (val_left.positions[coordinate] <= val_right.positions[coordinate]):
            sorted_arr.append(val_left)
            l_counter += 1
        else:
            sorted_arr.append(val_right)
            r_counter += 1
    
    while l_counter < len(left):
        sorted_arr.append(left[l_counter])
        l_counter += 1
        
    while r_counter < len(right):
        sorted_arr.append(right[r_counter])
        r_counter += 1
        
    return sorted_arr

def merge_sort_points(points, coordinate):
    size = len(points)
    
    if size <= 1:
        return points
    
    middle = int((size - 1) / 2) + 1
    
    left = merge_sort_points(points[:middle], coordinate) # Processando a metade da esquerda
    right = merge_sort_points(points[middle:], coordinate) # Processando a metade da direita
    
    return merge(left, right, coordinate)

p1 = Point([5, 1])
p2 = Point([1, 2])
p3 = Point([2, 3])
p4 = Point([4, 4])
p5 = Point([9, 5])
p6 = Point([3, 6])

l1 = [p2, p6, p3, p4, p1, p5]
l2 = [p1, p2, p3, p4, p5, p6]
l3 = [p2, p3, p6, p4, p1, p5]

merge_sort_points(l3, 1)"""

'def merge(left, right, coordinate):\n    l_counter = r_counter = 0\n    sorted_arr = []\n    \n    while l_counter < len(left) and r_counter < len(right):\n        val_left = left[l_counter]\n        val_right = right[r_counter]\n        \n        if (val_left.positions[coordinate] <= val_right.positions[coordinate]):\n            sorted_arr.append(val_left)\n            l_counter += 1\n        else:\n            sorted_arr.append(val_right)\n            r_counter += 1\n    \n    while l_counter < len(left):\n        sorted_arr.append(left[l_counter])\n        l_counter += 1\n        \n    while r_counter < len(right):\n        sorted_arr.append(right[r_counter])\n        r_counter += 1\n        \n    return sorted_arr\n\ndef merge_sort_points(points, coordinate):\n    size = len(points)\n    \n    if size <= 1:\n        return points\n    \n    middle = int((size - 1) / 2) + 1\n    \n    left = merge_sort_points(points[:middle], coordinate) # Processando a metade da esquerda\n    rig

In [4]:
from random import randrange

# Função utilitária que recebe um conjunto de pontos e, considerando uma determinada coordenada (também
# recebida como parâmetro), encontra o k-ésimo menor ponto desse conjunto. Para encontrar o ponto mediano
# de um conjunto de tamanho n, basta executar quickSelect com k = n/2
# Goodrich e Tamassia, Cap 9.2.1
def quickSelect(points, k, coordinate):
    if len(points) == 1:
        return points[0]
    
    random_elem = points[randrange(len(points))]
    
    lesser = []
    equal = []
    greater = []
    
    for elem in points:
        if elem.positions[coordinate] < random_elem.positions[coordinate]:
            lesser.append(elem)
        elif elem.positions[coordinate] == random_elem.positions[coordinate]:
            equal.append(elem)
        else:
            greater.append(elem)
    
    if k <= len(lesser):
        return quickSelect(lesser, k, coordinate)
    elif k <= (len(lesser) + len(equal)):
        return random_elem
    else:
        return quickSelect(greater,( k-len(lesser)-len(equal) ), coordinate)

# QuickSelect para conjuntos de inteiros
"""def quickSelect(S, k):
    if len(S) == 1:
        return S[0]
    
    random_elem = S[randrange(len(S))]
    
    lesser = []
    equal = []
    greater = []
    
    for elem in S:
        if elem < random_elem:
            lesser.append(elem)
        elif elem == random_elem:
            equal.append(elem)
        else:
            greater.append(elem)
    
    if k <= len(lesser):
        return quickSelect(lesser, k)
    elif k <= (len(lesser) + len(equal)):
        return random_elem
    else:
        return quickSelect(greater,( k-len(lesser)-len(equal) ))
        
S = [0,1,2,3,4,5,6,7,8,9]
S1 = [0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,2,2]
S2 = [5,8,0,1,7,3,6,9,4,2]

quickSelect(S2, 3)    
"""

'def quickSelect(S, k):\n    if len(S) == 1:\n        return S[0]\n    \n    random_elem = S[randrange(len(S))]\n    \n    lesser = []\n    equal = []\n    greater = []\n    \n    for elem in S:\n        if elem < random_elem:\n            lesser.append(elem)\n        elif elem == random_elem:\n            equal.append(elem)\n        else:\n            greater.append(elem)\n    \n    if k <= len(lesser):\n        return quickSelect(lesser, k)\n    elif k <= (len(lesser) + len(equal)):\n        return random_elem\n    else:\n        return quickSelect(greater,( k-len(lesser)-len(equal) ))\n        \nS = [0,1,2,3,4,5,6,7,8,9]\nS1 = [0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,2,2]\nS2 = [5,8,0,1,7,3,6,9,4,2]\n\nquickSelect(S2, 3)    \n'

In [5]:
p1 = Point([1, 1])
p2 = Point([2, 2])
p3 = Point([3, 3])
p4 = Point([4, 4])
p5 = Point([5, 5])
p6 = Point([6, 6])
p7 = Point([7, 7])
p8 = Point([8, 8])

pontos1 = [p1, p2, p3, p4, p5, p6, p7, p8]
pontos2 = [p3, p6, p1, p7, p2, p8, p5, p4]
pontos3 = [p8, p3, p1, p7, p2, p6, p4, p5]
quickSelect(pontos3, 4, 0)

[4, 4]

In [6]:
# Função utilitária que recebe um conjunto de pontos e, para uma determinada coordenada desses pontos,
# encontra o ponto mediano e separa o conjunto original em dois subconjuntos da "esquerda" e "direita" 
# de tal forma que, no subconjunto da esquerda, se encontram todos os pontos que são menores ou iguais que 
# o mediano na coordenada que está sendo considerada e, no subconjunto da direita, se encontram os 
# pontos estritamente maiores que o ponto mediano. (Esses elementos NÃO estão necessariamente ordenados)

def find_median_point_and_separate(points, coordinate):
    
    median_index = int((len(points) - 1) / 2) + 1
    median_point = quickSelect(points, median_index, coordinate)
    
    lesser_or_equal = []
    greater = []
    
    for elem in points:
        if elem.positions[coordinate] <= median_point.positions[coordinate]:
            lesser_or_equal.append(elem)
        else:
            greater.append(elem)
    
    return median_point, lesser_or_equal, greater

In [7]:
p1 = Point([1, 1])
p2 = Point([2, 2])
p3 = Point([3, 3])
p4 = Point([4, 4])
p5 = Point([5, 5])
p6 = Point([6, 6])
p7 = Point([7, 7])
p8 = Point([8, 8])

pontos1 = [p1, p2, p3, p4, p5, p6, p7, p8]
pontos2 = [p3]
pontos3 = [p8, p3, p1, p7, p2, p6, p4]
median_point, lteq, gt = find_median_point_and_separate(pontos1, 0)
print(median_point)
print(lteq)
print(gt)

[4, 4]
[[1, 1], [2, 2], [3, 3], [4, 4]]
[[5, 5], [6, 6], [7, 7], [8, 8]]


In [8]:
# Função utilitária para construir uma k-d Tree a partir de um conjunto de pontos k-dimensionais.
# Os pontos ficarão armazenados nos Nós folhas, sendo que os nós internos armazenarão apenas referencias
# dos valores usados para dividir o espaço em cortes ou hiperplanos de uma determinada dimensão dos pontos.
# Cada profundidade da árvore será computada seguindo uma ordem sequencial das dimensões dos pontos.
# Por exemplo, com pontos tridimensionais (x = 0, y = 1 e z = 2), a raíz da árvore verificará a
# dimensão 0, seus nos filhos verificarão a dimensão 1, seus netos verificarão a dimensão 2, seus
# bisnetos verificarão a dimensão 0 e assim por diante

def create_k_d_tree(points):
    return create_k_d_tree_recursive(points, 0)

def create_k_d_tree_recursive(points, depth):
    if (len(points) == 0):
        return None

    dimensions = len(points[0].positions)
    considered_coordinate = depth % dimensions
    
    if len(points) == 1:
        leaf = Node(True, considered_coordinate, points[0].positions[considered_coordinate], points[0])
        return leaf
    else:
        median_point, lteq, gt = find_median_point_and_separate(points, considered_coordinate)
        
        internal_node = Node(False, considered_coordinate, median_point.positions[considered_coordinate], None)
        
        internal_node.left = create_k_d_tree_recursive(lteq, depth+1)
        internal_node.right = create_k_d_tree_recursive(gt, depth+1)
        
        return internal_node

In [9]:
# Função utilitária auxiliar para fazer um caminhamento "inorder" em uma k-d-tree

def inorder(root):
    if root:
        inorder(root.left)
        
        if root.is_leaf:
            print(root.point)
        else:
            print(root.position)
            
        inorder(root.right)

In [10]:
p1 = Point([6.7, 5.3])
p2 = Point([1.8, 3.6])
p3 = Point([4.8, 1])
p4 = Point([7.6, 4.5])
p5 = Point([6.7, 4.3])
p6 = Point([3.4, 7])
p7 = Point([5.9, 3])
p8 = Point([4.6, 4.2])
p9 = Point([2, 8.9])
p10 = Point([3.7, 8.6])

points = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]

k_d_tree = create_k_d_tree(points)
inorder(k_d_tree)

[1.8, 3.6]
3.6
[3.4, 7]
3.4
[4.6, 4.2]
7
[2, 8.9]
2
[3.7, 8.6]
4.6
[4.8, 1]
1
[5.9, 3]
5.9
[6.7, 4.3]
4.3
[6.7, 5.3]
6.7
[7.6, 4.5]


In [11]:
p1 = Point([1, 3])
p2 = Point([2, 2])
p3 = Point([4, 4])
p4 = Point([5, 7])

points = [p1, p2, p3, p4]

k_d_tree = create_k_d_tree(points)
inorder(k_d_tree)

[2, 2]
2
[1, 3]
2
[4, 4]
4
[5, 7]


## Testes ------------------------------------

In [12]:
class BidirectionalNode:
    def __init__(self, is_leaf, considered_coordinate, position, point, parent, child_direction):
        """
        Controi um Nó para uma k-d Tree
        
        is_leaf: Booleano que indica se esse Nó é uma Folha na árvore ou se é um Nó interno 
        True ou False
        
        considered_coordinate: Qual coordenada dos pontos foi analisada para se construir esse Nó 
        0..n-1, onde n é a dimensão (qtde de coordenadas) dos pontos
        
        position: Posição ou valor do ponto na coordenada considerada
        Float
        OBS: Se 'point is not None' então position = point.positions[considered_coordinate]
        
        point: Se esse Nó não for interno (for folha), ele deve ter um 'point' como contúdo
        
        parent: Uma referencia para o pai desse BidirectionalNode, se houver.
        
        child_direction: Dado que esse BidirectionalNode tem um pai, indica se esse BidirectionalNode e um 
        filho da esquerda ou da direita ('LEFT' = Esquerda, 'RIGHT' = Direita, 'N' = Nao tem pai)
        """
        
        self.left = None
        self.right = None
        self.parent = parent
        self.child_direction = child_direction
        self.is_leaf = is_leaf 
        self.considered_coordinate = considered_coordinate
        self.position = position
        
        self.point = point
        
    def __str__(self):
        if self.is_leaf:
            return str(self.point)
        else:
            return str(self.position)
    
    def __repr__(self):
        if self.is_leaf:
            return str(self.point)
        else:
            return str(self.position)

In [13]:
class Heap:
    
    def __init__(self):
        self.values = []
        
    def push(self, val):
        self.values.append(val)
    
    def pop(self):
        last_index = len(self.values)
        if last_index < 0:
            return None
        
        return self.values.pop(last_index - 1)
    
    def is_empty(self):
        return len(self.values) == 0
    
    def print_values (self):
        print(self.values)

In [14]:
t = Heap()
t.print_values()
t.push(15)
t.print_values()
t.push(11)
t.print_values()
t.push(42)
t.print_values()
v1 = t.pop()
print(v1)
t.print_values()
v2 = t.pop()
print(v2)
t.print_values()
t.push(1)
t.print_values()

[]
[15]
[15, 11]
[15, 11, 42]
42
[15, 11]
11
[15]
[15, 1]


In [15]:
class Auxiliary_kdtree_instruction:
    
    def __init__ (self, node, initial_index, final_index, direction):
        self.node = node
        self.initial_index = initial_index
        self.final_index = final_index
        self.direction = direction # 'LEFT', 'RIGHT' ou 'NONE'

In [16]:
def create_k_d_tree_iterative(points):
    if (len(points) == 0):
        return None
    elif (len(points) == 1):
        leaf = Node(True, 0, points[0].positions[0], points[0])
        return leaf
    
    dimensions = len(points[0].positions)
    
    aux_heap = Heap()
    
    #root_median_index = int((len(points) - 1) / 2) + 1
    #root_median_point, root_lteq, root_gt = find_median_point_and_separate(points, 0)
    
    #concatenated_array = root_lteq + root_gt
    
    #initial_index = 0
    #final_index = len(points)
    
    #points[initial_index:final_index] = concatenated_array
    
    #root = BidirectionalNode(False, 0, root_median_point.positions[0], root_median_point, None, "N")
    dummy_node = BidirectionalNode(False, -1, None, None, None, None)
    
    """first_inst = Auxiliary_kdtree_instruction(root, initial_index, root_median_index, "LEFT", 0)
    second_inst = Auxiliary_kdtree_instruction(root, root_median_index, final_index, "RIGHT", 0)
    aux_heap.push(first_inst)
    aux_heap.push(second_inst)"""
    root_inst = Auxiliary_kdtree_instruction(dummy_node, 0, len(points), "NONE")
    aux_heap.push(root_inst)
    root_node = None
    
    while not aux_heap.is_empty():
        current_instruction = aux_heap.pop()
        
        initial_index = current_instruction.initial_index
        final_index = current_instruction.final_index
        parent_node = current_instruction.node
        new_considered_coordinate = (parent_node.considered_coordinate + 1) % dimensions
        
        print(initial_index)
        print(final_index)
        print(parent_node)
        print(new_considered_coordinate)
        
        if (initial_index + 1) == final_index:
            # Apenas um ponto. Uma folha!
            leaf = BidirectionalNode(True, new_considered_coordinate, points[initial_index].positions[new_considered_coordinate], points[initial_index], parent_node, current_instruction.direction)
            print("FOLHA: ", points[initial_index])
            if current_instruction.direction == "LEFT":
                parent_node.left = leaf
            elif current_instruction.direction == "RIGHT":
                parent_node.right = leaf
            #elif current_instruction.direction == "NONE":
                #Do nothing
            
        else:
            sub_array = points[initial_index:final_index]
            
            median_index = int((len(sub_array) - 1) / 2) + 1
            median_point, lteq, gt = find_median_point_and_separate(sub_array, new_considered_coordinate)
            concatenated_array = lteq + gt
            points[initial_index:final_index] = concatenated_array
            
            internal_node = BidirectionalNode(False, new_considered_coordinate, median_point.positions[new_considered_coordinate], None, parent_node, current_instruction.direction)
            print("INTERNO: ", median_point.positions[new_considered_coordinate])
            if current_instruction.direction == "LEFT":
                parent_node.left = internal_node
            elif current_instruction.direction == "RIGHT":
                parent_node.right = internal_node
            elif current_instruction.direction == "NONE":
                root_node = internal_node
            
            left_inst = Auxiliary_kdtree_instruction(internal_node, initial_index, (initial_index + median_index), "LEFT")
            right_inst = Auxiliary_kdtree_instruction(internal_node, (initial_index + median_index), final_index, "RIGHT")
            aux_heap.push(left_inst)
            aux_heap.push(right_inst)
        
        print(points)
        print("---------------------------------")
    
    print(root_node.left)
    print(root_node.right)
    return root_node
    """print(median_index)
    print(median_point)
    print(lteq)
    print(gt)
    print(concatenated_array)
    print(points)"""

In [17]:
p1 = Point([1, 3])

points = [p1]
k_d_tree = create_k_d_tree_iterative(points)
inorder(k_d_tree)

[1, 3]


In [18]:
p1 = Point([1, 3])
p2 = Point([2, 2])
p3 = Point([4, 4])
p4 = Point([5, 7])

points = [p1, p2, p3, p4]

k_d_tree = create_k_d_tree_iterative(points)
print("********************")
inorder(k_d_tree)

0
4
None
0
INTERNO:  2
[[1, 3], [2, 2], [4, 4], [5, 7]]
---------------------------------
2
4
2
1
INTERNO:  4
[[1, 3], [2, 2], [4, 4], [5, 7]]
---------------------------------
3
4
4
0
FOLHA:  [5, 7]
[[1, 3], [2, 2], [4, 4], [5, 7]]
---------------------------------
2
3
4
0
FOLHA:  [4, 4]
[[1, 3], [2, 2], [4, 4], [5, 7]]
---------------------------------
0
2
2
1
INTERNO:  2
[[2, 2], [1, 3], [4, 4], [5, 7]]
---------------------------------
1
2
2
0
FOLHA:  [1, 3]
[[2, 2], [1, 3], [4, 4], [5, 7]]
---------------------------------
0
1
2
0
FOLHA:  [2, 2]
[[2, 2], [1, 3], [4, 4], [5, 7]]
---------------------------------
2
4
********************
[2, 2]
2
[1, 3]
2
[4, 4]
4
[5, 7]


In [19]:
p1 = Point([6.7, 5.3])
p2 = Point([1.8, 3.6])
p3 = Point([4.8, 1])
p4 = Point([7.6, 4.5])
p5 = Point([6.7, 4.3])
p6 = Point([3.4, 7])
p7 = Point([5.9, 3])
p8 = Point([4.6, 4.2])
p9 = Point([2, 8.9])
p10 = Point([3.7, 8.6])

points = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]

k_d_tree = create_k_d_tree_iterative(points)
print("****************************")
inorder(k_d_tree)

0
10
None
0
INTERNO:  4.6
[[1.8, 3.6], [3.4, 7], [4.6, 4.2], [2, 8.9], [3.7, 8.6], [6.7, 5.3], [4.8, 1], [7.6, 4.5], [6.7, 4.3], [5.9, 3]]
---------------------------------
5
10
4.6
1
INTERNO:  4.3
[[1.8, 3.6], [3.4, 7], [4.6, 4.2], [2, 8.9], [3.7, 8.6], [4.8, 1], [6.7, 4.3], [5.9, 3], [6.7, 5.3], [7.6, 4.5]]
---------------------------------
8
10
4.3
0
INTERNO:  6.7
[[1.8, 3.6], [3.4, 7], [4.6, 4.2], [2, 8.9], [3.7, 8.6], [4.8, 1], [6.7, 4.3], [5.9, 3], [6.7, 5.3], [7.6, 4.5]]
---------------------------------
9
10
6.7
1
FOLHA:  [7.6, 4.5]
[[1.8, 3.6], [3.4, 7], [4.6, 4.2], [2, 8.9], [3.7, 8.6], [4.8, 1], [6.7, 4.3], [5.9, 3], [6.7, 5.3], [7.6, 4.5]]
---------------------------------
8
9
6.7
1
FOLHA:  [6.7, 5.3]
[[1.8, 3.6], [3.4, 7], [4.6, 4.2], [2, 8.9], [3.7, 8.6], [4.8, 1], [6.7, 4.3], [5.9, 3], [6.7, 5.3], [7.6, 4.5]]
---------------------------------
5
8
4.3
0
INTERNO:  5.9
[[1.8, 3.6], [3.4, 7], [4.6, 4.2], [2, 8.9], [3.7, 8.6], [4.8, 1], [5.9, 3], [6.7, 4.3], [6.7, 5.3], [7.6,

## Fim dos Testes ------------------------------------

In [20]:
# Função utilitária que computa a distância euclidiana entre dois pointos. Note que, para economizar
# processamento, não tiramos a raíz quadrada durante o cálculo, logo, a distância retornada estará
# elevada ao quadrado

def euclidean_distance(p1, p2):
    dimensions = len(p1.positions)
    
    sum = 0
    
    for coord in range(dimensions):
        val_p1 = p1.positions[coord]
        val_p2 = p2.positions[coord]
        
        delta = val_p2 - val_p1
        sum += (delta**2)
    # return sqrt(sum)
    return sum

In [21]:
p1 = Point([1, 1])
p2 = Point([2, 2])
p3 = Point([1, 3])
p4 = Point([0, 0])
p5 = Point([8, 6])
p6 = Point([4, 2, 8])
p7 = Point([2, 9, 2])

print(euclidean_distance(p1, p2))
print(euclidean_distance(p1, p3))
print(euclidean_distance(p2, p3))
print(euclidean_distance(p4, p5))
print(euclidean_distance(p6, p7))

2
4
2
100
89


In [22]:
# Classe utilitária que recebe uma k-d-tree e um ponto e encontra o vizinho mais próximo desse ponto
# que esteja nessa árvore

class NearestNeighbour:

    def search_nearest_neighbour(self, k_d_tree_root, point):
        self.current_best = None
        self.k_d_tree_root = k_d_tree_root
        self.point = point
        self.search_nearest_neighbour_recursive(self.k_d_tree_root)
        return self.current_best
        
    def search_nearest_neighbour_recursive(self, k_d_tree_node):
        if k_d_tree_node.is_leaf:
            if self.current_best is None:
                self.current_best = k_d_tree_node.point
            else:
                current_node_distance = euclidean_distance(k_d_tree_node.point, self.point)
                current_best_distance = euclidean_distance(self.current_best, self.point)
                if current_node_distance < current_best_distance:
                    self.current_best = k_d_tree_node.point
        else:
            recursive_direction = ""
            
            if self.point.positions[k_d_tree_node.considered_coordinate] <= k_d_tree_node.position:
                recursive_direction = "LEFT"
                self.search_nearest_neighbour_recursive(k_d_tree_node.left)
            else:
                recursive_direction = "RIGHT"
                self.search_nearest_neighbour_recursive(k_d_tree_node.right)
                
            #-----------------------------------------
            #Verifying if its necessary to check the other subtree
            
            #cb_pos_val = self.current_best.positions[k_d_tree_node.considered_coordinate]
            distance_between_point_and_current_best = euclidean_distance(self.current_best, self.point)
            point_pos_val = self.point.positions[k_d_tree_node.considered_coordinate]
            hyperplane_split_value = k_d_tree_node.position
            
            if ((hyperplane_split_value - point_pos_val)**2) <= distance_between_point_and_current_best:
                # The hipersphere intersects the hiperplane, therefore, there could be better (closer) points
                # https://en.wikipedia.org/wiki/K-d_tree
                
                if recursive_direction == "LEFT":
                    self.search_nearest_neighbour_recursive(k_d_tree_node.right)
                else:
                    self.search_nearest_neighbour_recursive(k_d_tree_node.left)

In [23]:
p1 = Point([1, 3])
p2 = Point([2, 2])
p3 = Point([4, 4])
p4 = Point([5, 7])

points = [p1, p2, p3, p4]

k_d_tree = create_k_d_tree(points)
#inorder(k_d_tree)

px = Point([1, 1])
py = Point([5, 9])
pz = Point([3, 3])
pk = Point([5, 5])

nn = NearestNeighbour()
print(nn.search_nearest_neighbour(k_d_tree, px))
print(nn.search_nearest_neighbour(k_d_tree, py))
print(nn.search_nearest_neighbour(k_d_tree, pz))
print(nn.search_nearest_neighbour(k_d_tree, pk))

[2, 2]
[5, 7]
[4, 4]
[4, 4]


In [24]:
p1 = Point([1, 3])
p2 = Point([1, 8])
p3 = Point([2, 2])
p4 = Point([2, 10])
p5 = Point([3, 6])
p6 = Point([4, 1])
p7 = Point([5, 4])
p8 = Point([6, 8])
p9 = Point([7, 4])
p10 = Point([7, 7])
p11 = Point([8, 2])
p12 = Point([8, 5])
p13 = Point([9, 9])

points = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13]

k_d_tree = create_k_d_tree(points)
#inorder(k_d_tree)

px = Point([4, 8])

nn = NearestNeighbour()
print(nn.search_nearest_neighbour(k_d_tree, px))

[6, 8]


In [25]:
# Classe utilitária que recebe uma k-d-tree, um inteiro k e um ponto e encontra os k vizinhos 
# mais próximos desse ponto que estejam nessa árvore

class KNearestNeighbours:

    def search_k_nearest_neighbours(self, k, k_d_tree_root, point):
        self.k = k
        self.current_best_priority_queue = []
        self.k_d_tree_root = k_d_tree_root
        self.point = point
        self.search_k_nearest_neighbours_recursive(self.k_d_tree_root)
        return self.current_best_priority_queue
    
    def queue_is_full(self):
        return len(self.current_best_priority_queue) == self.k
    
    def find_farthest_best_point(self):
        farthest_distance = 0
        index_of_farthest_best_point = -1
        farthest_best_point = None

        counter = 0

        for one_best_point in self.current_best_priority_queue:
            one_best_point_distance = euclidean_distance(one_best_point, self.point)

            if one_best_point_distance > farthest_distance:
                farthest_distance = one_best_point_distance
                index_of_farthest_best_point = counter
                farthest_best_point = one_best_point

            counter += 1
        
        return farthest_best_point, index_of_farthest_best_point, farthest_distance
    
    def check_value_and_update_queue(self, k_d_tree_node):
        if not self.queue_is_full():
            self.current_best_priority_queue.append(k_d_tree_node.point)
        else:
            
            farthest_best_point, index_of_farthest_best_point, farthest_distance = self.find_farthest_best_point()
            
            current_node_distance = euclidean_distance(k_d_tree_node.point, self.point)
            
            if current_node_distance < farthest_distance:
                self.current_best_priority_queue.pop(index_of_farthest_best_point)
                self.current_best_priority_queue.append(k_d_tree_node.point)
        
    def search_k_nearest_neighbours_recursive(self, k_d_tree_node):
        if k_d_tree_node is None:
            return
        
        if k_d_tree_node.is_leaf:
            self.check_value_and_update_queue(k_d_tree_node)
        else:
            recursive_direction = ""
            
            if self.point.positions[k_d_tree_node.considered_coordinate] <= k_d_tree_node.position:
                recursive_direction = "LEFT"
                self.search_k_nearest_neighbours_recursive(k_d_tree_node.left)
            else:
                recursive_direction = "RIGHT"
                self.search_k_nearest_neighbours_recursive(k_d_tree_node.right)
            
            #-----------------------------------------
            #Verifying if its necessary to check the other subtree 
            
            farthest_best_point, index_of_farthest_best_point, farthest_distance = self.find_farthest_best_point()
            #OBS: 'farthest_distance' is the radius of the hipersphere
            
            #Instead of using the following value, use 'farthest_distance' to compare
            #farthest_best_point_pos_val = farthest_best_point.positions[k_d_tree_node.considered_coordinate]
            point_pos_val = self.point.positions[k_d_tree_node.considered_coordinate]
            hyperplane_split_value = k_d_tree_node.position
            
            if (not self.queue_is_full()) or (((hyperplane_split_value - point_pos_val)**2) <= farthest_distance):
                if recursive_direction == "LEFT":
                    self.search_k_nearest_neighbours_recursive(k_d_tree_node.right)
                else:
                    self.search_k_nearest_neighbours_recursive(k_d_tree_node.left)

In [26]:
p1 = Point([1, 3])
p2 = Point([2, 2])
p3 = Point([4, 4])
p4 = Point([5, 7])

points = [p1, p2, p3, p4]

k_d_tree = create_k_d_tree(points)
#inorder(k_d_tree)

px = Point([1, 1])
py = Point([5, 9])
pz = Point([3, 3])
pk = Point([5, 5])

knn = KNearestNeighbours()
print(knn.search_k_nearest_neighbours(1, k_d_tree, px))
print(knn.search_k_nearest_neighbours(1, k_d_tree, py))
print(knn.search_k_nearest_neighbours(1, k_d_tree, pz))
print(knn.search_k_nearest_neighbours(1, k_d_tree, pk))
print("---")
print(knn.search_k_nearest_neighbours(2, k_d_tree, px))
print(knn.search_k_nearest_neighbours(2, k_d_tree, py))
print(knn.search_k_nearest_neighbours(2, k_d_tree, pz))
print(knn.search_k_nearest_neighbours(2, k_d_tree, pk))

[[2, 2]]
[[5, 7]]
[[4, 4]]
[[4, 4]]
---
[[2, 2], [1, 3]]
[[5, 7], [4, 4]]
[[4, 4], [2, 2]]
[[5, 7], [4, 4]]


In [27]:
p1 = Point([1, 3])
p2 = Point([1, 8])
p3 = Point([2, 2])
p4 = Point([2, 10])
p5 = Point([3, 6])
p6 = Point([4, 1])
p7 = Point([5, 4])
p8 = Point([6, 8])
p9 = Point([7, 4])
p10 = Point([7, 7])
p11 = Point([8, 2])
p12 = Point([8, 5])
p13 = Point([9, 9])

points = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13]

k_d_tree = create_k_d_tree(points)
#inorder(k_d_tree)

px = Point([4, 8])

knn = KNearestNeighbours()

print(knn.search_k_nearest_neighbours(1, k_d_tree, px))
print(knn.search_k_nearest_neighbours(2, k_d_tree, px))
print(knn.search_k_nearest_neighbours(3, k_d_tree, px))
print(knn.search_k_nearest_neighbours(4, k_d_tree, px))
print(knn.search_k_nearest_neighbours(5, k_d_tree, px))

[[6, 8]]
[[3, 6], [6, 8]]
[[3, 6], [2, 10], [6, 8]]
[[3, 6], [1, 8], [2, 10], [6, 8]]
[[3, 6], [1, 8], [2, 10], [6, 8], [7, 7]]


In [28]:
import pandas as pd
import random
import math

class KNNClassifierAlgorithm:
    
    def __init__(self, original_dataset):
        self.original_dataset = original_dataset
        
        #------------ Creating the set of points based on the original Dataset ------------
        self.set_of_all_points = []
        #number_of_columns = original_dataset.shape[1]
        #X = dataset.iloc[:, 0:(number_of_columns-1)]     # The first columns in our Dataset, except the last one
        #y = dataset.iloc[:, (number_of_columns-1)]       # The last column
        for index, row in original_dataset.iterrows():
            row_as_array = row.to_numpy()
            number_of_columns = len(row_as_array)
            x = row_as_array[0:(number_of_columns-1)]     # The first columns in our Dataset, except the last one
            y = row_as_array[(number_of_columns-1)]       # The last column
            
            p = Point(x, y)
            self.set_of_all_points.append(p)
        
        #------------ Separating Train and Test subsets ------------
        random.shuffle(self.set_of_all_points)
        train_proportion = 0.70
        index_separator = int(len(self.set_of_all_points) * train_proportion)
        
        self.Train = self.set_of_all_points[:index_separator]
        self.Test = self.set_of_all_points[index_separator:]
        
        #------------ Computing the k-d-tree with the Train subset ------------
        self.k_d_tree = create_k_d_tree(self.Train)
        
        #------------ Computing the optimal 'k' value------------
        
        optimal_k = int(math.sqrt(len(self.set_of_all_points)))
        if optimal_k % 2 == 0:
            optimal_k -= 1
        #optimal_k = 10
        
        #------------ Predicting the classification with the Test subset ------------
        self.hits = 0
        self.misses = 0
        
        knn = KNearestNeighbours()
        
        for test_point in self.Test:
            k_nearest_neighbours = knn.search_k_nearest_neighbours(optimal_k, self.k_d_tree, test_point)
            
            class_dict = {}
            
            for nearest_neighbour in k_nearest_neighbours:
                if nearest_neighbour.classification in class_dict:
                    class_dict[nearest_neighbour.classification] += 1
                else:
                    class_dict[nearest_neighbour.classification] = 1
            
            majority_class = None
            majority_number = 0

            for k, v in class_dict.items():
                if v > majority_number:
                    majority_number = v
                    majority_class = k
            
            #print("Predicted class: ", majority_class, " -- Actual class: ", test_point.classification)
            test_point.predicted_classification = majority_class
            
            #Comparing the predicted value with the actual real value
            if majority_class == test_point.classification:
                self.hits += 1
            else:
                self.misses += 1
            
        #------------ Computing the accuracy score ------------
        self.accuracy_score = self.hits / len(self.Test)
        
        #------------ Computing the precision and recall scores ------------
        
        class_dict_with_true_and_false_positives = {}
            
        for test_point in self.Test:
            if test_point.predicted_classification not in class_dict_with_true_and_false_positives:
                class_dict_with_true_and_false_positives[test_point.predicted_classification] = [0, 0]
                # [num of true positives, num of false positives]
            
            if test_point.predicted_classification == test_point.classification:
                true_positives = class_dict_with_true_and_false_positives[test_point.predicted_classification][0]
                true_positives += 1
                class_dict_with_true_and_false_positives[test_point.predicted_classification][0] = true_positives
            else:
                false_positives = class_dict_with_true_and_false_positives[test_point.predicted_classification][1]
                false_positives += 1
                class_dict_with_true_and_false_positives[test_point.predicted_classification][1] = false_positives
                
        self.precision_and_recall_per_class = []
        
        for k, v in class_dict_with_true_and_false_positives.items():
            precision = v[0] / (v[0] + v[1])
            recall = v[0] / self.num_of_real_occurrences_of_class(k)
            
            precision_and_recall = {
                "class" : k,
                "precision" : precision,
                "recall" : recall
            }
            
            self.precision_and_recall_per_class.append(precision_and_recall)
            
    def num_of_real_occurrences_of_class(self, point_class):
        num = 0
        for test_point in self.Test:
            if test_point.classification == point_class:
                num += 1
        
        return num
        
    def print_info(self):
        print("----- Train subset: (", len(self.Train), " elements) -----")
        for train_point in self.Train:
            print(train_point)
        print()
        print("----- Test subset: (", len(self.Test), " elements, with predicted value) -----")
        for test_point in self.Test:
            print(test_point)
        print()
        print("----- Hits: ", self.hits, " -----")
        print("----- Misses: ", self.misses, " -----")
        print("----- Accuracy score: ", self.accuracy_score, " -----")
        print("----- Precision and Recall per class: -----")
        for precision_and_recall in self.precision_and_recall_per_class:
            print("For class (", precision_and_recall["class"], ") , the precision is ", precision_and_recall["precision"], " and the recall is ", precision_and_recall["recall"])
        

Até então, praticamente todas as funções utilitárias que precisaremos já foram implementadas. Agora, podemos realmente começar a trabalhar com um dataset real e fazer a previsão e classificação com kNN.

In [29]:
def load_dataset(path, max_size = 1000):
    # OBS: Tive SERÍSSIMOS problemas com datasets muito grandes. O Kernel do meu Jupyter estava morrendo
    # a todo momento quando se tentava gerar a k-d-tree para alguns datasets maiores, logo, para 
    # contornar esse problema, eu carrego o dataset e, se ele possuir, por padrão, mais que 1000 entradas,
    # eu randomizo suas linhas e pego suas 1000 primeiras entradas para serem usadas.
    # (Nos datasets mais pesados, tive que diminuir a quantidade de linhas processadas para 100!!!)
    
    dataset = pd.read_csv(path)
    
    if len(dataset) > max_size:
        dataset = dataset.sample(frac=1)
        dataset = dataset.head(max_size)
    
    return dataset

### banana

In [30]:
dataset = load_dataset('datasets_tratados/banana.csv')
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 700  elements) -----
[0.552, -0.244]-(1.0)
[0.396, 1.81]-(1.0)
[-0.816, 0.562]-(-1.0)
[-1.45, -0.0697]-(1.0)
[0.905, -0.994]-(1.0)
[-1.81, -0.994]-(1.0)
[-0.978, -1.05]-(1.0)
[-0.681, -0.586]-(1.0)
[0.923, 0.166]-(1.0)
[0.0762, -1.27]-(-1.0)
[0.553, -0.159]-(1.0)
[1.07, 1.53]-(-1.0)
[1.14, 0.866]-(-1.0)
[2.71, 1.35]-(1.0)
[-1.2, -0.826]-(1.0)
[2.34, -1.25]-(-1.0)
[0.166, -1.25]-(-1.0)
[-1.98, 0.345]-(-1.0)
[0.105, -0.208]-(1.0)
[0.878, -1.38]-(1.0)
[-0.339, 0.806]-(1.0)
[0.762, 0.125]-(1.0)
[0.579, -1.02]-(1.0)
[-0.784, -1.05]-(-1.0)
[1.87, -0.239]-(-1.0)
[-0.0767, 0.000168]-(1.0)
[0.00665, 0.0118]-(1.0)
[-0.0338, 1.52]-(-1.0)
[-0.124, -0.95]-(-1.0)
[0.778, 1.44]-(1.0)
[1.38, 0.749]-(-1.0)
[0.196, 1.76]-(-1.0)
[0.701, 0.0237]-(1.0)
[-0.988, -0.191]-(1.0)
[-1.65, -0.0692]-(1.0)
[0.282, -0.761]-(1.0)
[0.386, -0.997]-(-1.0)
[-1.12, -0.315]-(1.0)
[-1.17, 0.156]-(1.0)
[0.375, 1.24]-(-1.0)
[-0.611, -1.28]-(-1.0)
[1.09, -1.59]-(1.0)
[-0.961, 0.755]-(-1.0)
[1.49, -0.886]-

### ecoli1

In [31]:
dataset = load_dataset('datasets_tratados/ecoli1.csv')
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 188  elements) -----
[0.72, 0.86, 0.48, 0.5, 0.17, 0.55, 0.21]-(negative)
[0.34, 0.35, 0.48, 0.5, 0.51, 0.49, 0.56]-(negative)
[0.75, 0.76, 0.48, 0.5, 0.83, 0.57, 0.3]-(negative)
[0.79, 0.41, 0.48, 0.5, 0.66, 0.81, 0.83]-(negative)
[0.07, 0.4, 0.48, 0.5, 0.54, 0.35, 0.44]-(negative)
[0.38, 0.4, 0.48, 0.5, 0.63, 0.25, 0.35]-(negative)
[0.16, 0.51, 0.48, 0.5, 0.33, 0.39, 0.48]-(positive)
[0.44, 0.56, 0.48, 0.5, 0.5, 0.46, 0.54]-(negative)
[0.74, 0.78, 0.48, 0.5, 0.75, 0.54, 0.15]-(negative)
[0.61, 0.52, 0.48, 0.5, 0.54, 0.67, 0.52]-(positive)
[0.83, 0.48, 0.48, 0.5, 0.65, 0.76, 0.79]-(negative)
[0.7, 0.71, 0.48, 0.5, 0.42, 0.84, 0.85]-(negative)
[0.67, 0.55, 1.0, 0.5, 0.66, 0.58, 0.16]-(negative)
[0.41, 0.51, 0.48, 0.5, 0.58, 0.2, 0.31]-(negative)
[0.17, 0.38, 0.48, 0.5, 0.45, 0.42, 0.5]-(negative)
[0.56, 0.68, 0.48, 0.5, 0.77, 0.36, 0.45]-(negative)
[0.44, 0.49, 0.48, 0.5, 0.39, 0.38, 0.4]-(negative)
[0.67, 0.81, 0.48, 0.5, 0.54, 0.49, 0.23]-(negative)
[0.23, 0.33,

### glass0

In [32]:
dataset = load_dataset('datasets_tratados/glass0.csv')
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 120  elements) -----
[1.52739214, 11.0226, 0.0, 0.74903, 73.0804, 0.0, 14.96336, 0.0, 0.0]-(negative)
[1.5167311, 12.7915, 3.52016, 1.53869, 73.3604, 0.65826, 7.9048, 0.0, 0.0]-(negative)
[1.51652608, 11.94695, 0.0, 1.1888, 75.1804, 2.70135, 8.927, 0.0, 0.0]-(negative)
[1.51976084, 13.80895, 3.57853, 1.32041, 71.7196, 0.11799, 8.66876, 0.68985, 0.0]-(positive)
[1.51837126, 13.1373, 2.84217, 1.27868, 72.8508, 0.55269, 9.06688, 0.0, 0.0]-(positive)
[1.5215149, 11.02925, 1.71069, 1.56116, 73.4388, 0.57753, 11.617, 0.0, 0.0]-(negative)
[1.52101374, 13.6427, 4.49, 1.09892, 71.7812, 0.0621, 8.75484, 0.0, 0.0]-(positive)
[1.51925968, 13.19715, 3.33158, 1.27868, 72.358, 0.60237, 9.1422, 0.0, 0.0561]-(positive)
[1.51618438, 13.52965, 3.55159, 1.53869, 72.9908, 0.39123, 7.77568, 0.0, 0.0]-(positive)
[1.51595658, 13.0176, 3.56057, 1.53869, 73.1084, 0.72036, 7.9048, 0.0, 0.0]-(negative)
[1.53393, 12.2994, 0.0, 0.99941, 70.1628, 0.11799, 16.19, 0.0, 0.1224]-(negative)
[1.51693

### haberman

In [33]:
dataset = load_dataset('datasets_tratados/haberman.csv', 50)
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 35  elements) -----
[73.0, 62.0, 0.0]-(negative)
[54.0, 62.0, 0.0]-(negative)
[49.0, 64.0, 10.0]-(positive)
[49.0, 62.0, 0.0]-(negative)
[66.0, 58.0, 1.0]-(negative)
[45.0, 64.0, 0.0]-(negative)
[41.0, 58.0, 0.0]-(negative)
[36.0, 69.0, 0.0]-(negative)
[73.0, 68.0, 0.0]-(negative)
[66.0, 58.0, 0.0]-(negative)
[57.0, 64.0, 0.0]-(negative)
[58.0, 61.0, 1.0]-(negative)
[50.0, 63.0, 1.0]-(negative)
[50.0, 59.0, 0.0]-(negative)
[31.0, 59.0, 2.0]-(negative)
[51.0, 59.0, 3.0]-(positive)
[69.0, 65.0, 0.0]-(negative)
[38.0, 60.0, 1.0]-(negative)
[59.0, 63.0, 0.0]-(negative)
[56.0, 60.0, 0.0]-(negative)
[72.0, 58.0, 0.0]-(negative)
[60.0, 59.0, 17.0]-(positive)
[45.0, 59.0, 14.0]-(negative)
[59.0, 64.0, 1.0]-(negative)
[59.0, 64.0, 4.0]-(negative)
[74.0, 65.0, 3.0]-(positive)
[70.0, 66.0, 14.0]-(negative)
[55.0, 69.0, 22.0]-(negative)
[66.0, 61.0, 13.0]-(positive)
[33.0, 58.0, 10.0]-(negative)
[38.0, 62.0, 3.0]-(negative)
[52.0, 66.0, 4.0]-(positive)
[54.0, 67.0, 46.0]-(neg

### iris0

In [34]:
dataset = load_dataset('datasets_tratados/iris0.csv', 50)
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 35  elements) -----
[5.1, 3.3, 1.7, 0.5]-(positive)
[6.3, 2.9, 5.6, 1.8]-(negative)
[5.1, 3.4, 1.5, 0.2]-(positive)
[5.9, 3.2, 4.8, 1.8]-(negative)
[7.4, 2.8, 6.1, 1.9]-(negative)
[5.5, 2.4, 3.8, 1.1]-(negative)
[5.7, 2.9, 4.2, 1.3]-(negative)
[6.6, 3.0, 4.4, 1.4]-(negative)
[5.2, 2.7, 3.9, 1.4]-(negative)
[5.5, 2.6, 4.4, 1.2]-(negative)
[7.7, 3.0, 6.1, 2.3]-(negative)
[6.4, 3.2, 5.3, 2.3]-(negative)
[5.0, 3.0, 1.6, 0.2]-(positive)
[4.9, 3.0, 1.4, 0.2]-(positive)
[7.2, 3.2, 6.0, 1.8]-(negative)
[5.1, 2.5, 3.0, 1.1]-(negative)
[5.8, 2.7, 5.1, 1.9]-(negative)
[6.2, 3.4, 5.4, 2.3]-(negative)
[4.6, 3.1, 1.5, 0.2]-(positive)
[6.7, 3.1, 4.4, 1.4]-(negative)
[6.9, 3.1, 5.1, 2.3]-(negative)
[7.7, 2.6, 6.9, 2.3]-(negative)
[6.2, 2.8, 4.8, 1.8]-(negative)
[7.1, 3.0, 5.9, 2.1]-(negative)
[5.4, 3.0, 4.5, 1.5]-(negative)
[4.8, 3.0, 1.4, 0.1]-(positive)
[6.3, 3.3, 4.7, 1.6]-(negative)
[5.8, 2.7, 4.1, 1.0]-(negative)
[5.6, 2.5, 3.9, 1.1]-(negative)
[4.9, 3.1, 1.5, 0.1]-(positive

### new-thyroid2

In [35]:
dataset = load_dataset('datasets_tratados/new-thyroid2.csv', 50)
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 35  elements) -----
[110, 11.3, 2.3, 0.9, 3.3]-(negative)
[126, 9.4, 2.3, 1.0, 4.0]-(negative)
[129, 1.5, 0.6, 12.5, 2.9]-(negative)
[141, 5.6, 1.8, 9.2, 14.4]-(negative)
[110, 10.4, 1.8, 1.0, 2.3]-(negative)
[109, 8.9, 1.7, 1.0, 0.9]-(negative)
[101, 7.1, 2.2, 0.8, 2.2]-(negative)
[111, 16.0, 2.1, 0.9, -0.1]-(positive)
[117, 12.2, 1.9, 1.2, 3.9]-(negative)
[122, 11.8, 2.7, 1.7, 2.3]-(negative)
[108, 10.4, 2.1, 1.3, 2.4]-(negative)
[89, 21.8, 7.1, 0.7, -0.1]-(positive)
[102, 7.6, 1.8, 2.0, 2.5]-(negative)
[104, 9.6, 1.1, 1.3, 0.8]-(negative)
[108, 7.1, 1.3, 1.6, 2.2]-(negative)
[114, 8.4, 1.6, 1.6, -0.2]-(negative)
[102, 9.5, 1.4, 1.1, 1.6]-(negative)
[115, 6.3, 1.2, 4.7, 14.4]-(negative)
[80, 23.0, 10.0, 0.9, -0.1]-(positive)
[105, 17.4, 1.6, 0.3, 0.4]-(positive)
[105, 9.5, 1.8, 1.6, 3.6]-(negative)
[100, 10.5, 2.4, 0.9, 1.9]-(negative)
[123, 8.1, 2.3, 1.0, 5.1]-(negative)
[98, 9.1, 1.4, 1.9, -0.3]-(negative)
[118, 3.6, 1.5, 11.6, 48.8]-(negative)
[98, 8.6, 1.6, 

### page-blocks0

In [36]:
dataset = load_dataset('datasets_tratados/page-blocks0.csv', 50)
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 35  elements) -----
[6, 47, 282, 7.833, 0.177, 0.45, 1.61, 50, 127, 31]-(negative)
[9, 137, 1233, 15.222, 0.235, 0.693, 1.7, 290, 854, 171]-(negative)
[8, 11, 88, 1.375, 0.477, 0.966, 2.0, 42, 85, 21]-(negative)
[7, 7, 49, 1.0, 0.347, 0.98, 2.13, 17, 48, 8]-(negative)
[9, 369, 3321, 41.0, 0.329, 0.639, 2.67, 1094, 2122, 409]-(negative)
[2, 22, 44, 11.0, 0.75, 0.75, 33.0, 33, 33, 1]-(positive)
[8, 8, 64, 1.0, 0.406, 0.953, 2.17, 26, 61, 12]-(negative)
[9, 112, 1008, 12.444, 0.213, 0.835, 1.59, 215, 842, 135]-(negative)
[8, 75, 600, 9.375, 0.375, 0.86, 2.05, 225, 516, 110]-(negative)
[7, 19, 133, 2.714, 0.451, 0.97, 2.73, 60, 129, 22]-(negative)
[9, 32, 288, 3.556, 0.413, 0.993, 1.72, 119, 286, 69]-(negative)
[9, 40, 360, 4.444, 0.275, 0.875, 1.6, 99, 315, 62]-(negative)
[10, 53, 530, 5.3, 0.215, 0.781, 1.44, 114, 414, 79]-(negative)
[10, 125, 1250, 12.5, 0.456, 0.862, 2.92, 570, 1078, 195]-(negative)
[6, 17, 102, 2.833, 0.245, 0.804, 1.39, 25, 82, 18]-(negative)
[1

### pimaImb

In [37]:
dataset = load_dataset('datasets_tratados/pimaImb.csv', 50)
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 35  elements) -----
[0.0, 134.0, 58.0, 20.0, 291.0, 26.4, 0.352, 21.0]-(negative)
[1.0, 84.0, 64.0, 23.0, 115.0, 36.9, 0.471, 28.0]-(negative)
[0.0, 111.0, 65.0, 0.0, 0.0, 24.6, 0.66, 31.0]-(negative)
[3.0, 108.0, 62.0, 24.0, 0.0, 26.0, 0.223, 25.0]-(negative)
[2.0, 112.0, 86.0, 42.0, 160.0, 38.4, 0.246, 28.0]-(negative)
[6.0, 105.0, 80.0, 28.0, 0.0, 32.5, 0.878, 26.0]-(negative)
[0.0, 91.0, 80.0, 0.0, 0.0, 32.4, 0.601, 27.0]-(negative)
[3.0, 74.0, 68.0, 28.0, 45.0, 29.7, 0.293, 23.0]-(negative)
[1.0, 124.0, 60.0, 32.0, 0.0, 35.8, 0.514, 21.0]-(negative)
[7.0, 124.0, 70.0, 33.0, 215.0, 25.5, 0.161, 37.0]-(negative)
[4.0, 110.0, 92.0, 0.0, 0.0, 37.6, 0.191, 30.0]-(negative)
[5.0, 106.0, 82.0, 30.0, 0.0, 39.5, 0.286, 38.0]-(negative)
[8.0, 188.0, 78.0, 0.0, 0.0, 47.9, 0.137, 43.0]-(positive)
[3.0, 142.0, 80.0, 15.0, 0.0, 32.4, 0.2, 63.0]-(negative)
[0.0, 120.0, 74.0, 18.0, 63.0, 30.5, 0.285, 26.0]-(negative)
[2.0, 99.0, 52.0, 15.0, 94.0, 24.6, 0.637, 21.0]-(negative

### shuttle-c0-vs-c4

In [38]:
dataset = load_dataset('datasets_tratados/shuttle-c0-vs-c4.csv', 50)
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 35  elements) -----
[39, 0, 86, 5, 38, 0, 47, 48, 0]-(negative)
[42, 3, 96, 0, 42, 0, 54, 55, 2]-(negative)
[41, 0, 82, 0, 38, -18, 41, 43, 2]-(negative)
[44, 0, 91, -5, 44, 0, 47, 47, 0]-(negative)
[47, 0, 78, -7, 46, 0, 31, 32, 0]-(negative)
[56, 1, 86, 0, 56, 8, 30, 29, 0]-(negative)
[40, 4, 90, 0, 38, 0, 49, 51, 2]-(negative)
[51, 2, 83, -1, 50, -1, 32, 33, 2]-(negative)
[43, -3, 79, 0, 42, -16, 35, 37, 2]-(negative)
[37, 0, 80, 0, 20, -3, 43, 59, 16]-(negative)
[41, -3, 76, 0, 38, -19, 35, 37, 2]-(negative)
[43, -2, 77, 0, 44, 18, 34, 34, 0]-(negative)
[41, 4, 90, 7, 42, 15, 48, 48, 0]-(negative)
[53, 1, 78, 0, 52, -2, 25, 26, 2]-(negative)
[37, 0, 77, -2, 34, -898, 41, 44, 2]-(negative)
[55, 0, 83, 0, 56, 31, 27, 26, 0]-(negative)
[38, -1, 91, -2, 6, -7, 53, 86, 32]-(negative)
[85, 0, 88, 0, 2, 17, 3, 86, 82]-(positive)
[37, 0, 81, -7, 30, 0, 45, 50, 6]-(negative)
[53, 0, 85, -3, 54, 0, 32, 31, 0]-(negative)
[47, 0, 81, 3, 46, 0, 34, 34, 0]-(negative)
[42, 0

### vehicle3

In [39]:
dataset = load_dataset('datasets_tratados/vehicle3.csv', 50)
knnClassifier = KNNClassifierAlgorithm(dataset)
knnClassifier.print_info()

----- Train subset: ( 35  elements) -----
[85, 45, 82, 133, 56, 11, 159, 43, 20, 156, 170, 362, 173, 76, 10, 21, 183, 193]-(negative)
[96, 49, 98, 187, 59, 6, 213, 31, 24, 152, 228, 680, 210, 77, 8, 28, 188, 189]-(negative)
[89, 38, 77, 161, 62, 7, 149, 45, 19, 129, 174, 327, 153, 71, 6, 21, 188, 193]-(negative)
[93, 36, 63, 139, 57, 8, 132, 50, 18, 136, 158, 260, 121, 67, 3, 27, 193, 201]-(negative)
[90, 39, 57, 114, 48, 7, 135, 51, 18, 139, 155, 261, 151, 85, 12, 8, 183, 182]-(negative)
[103, 57, 105, 221, 69, 11, 218, 30, 24, 173, 226, 706, 250, 73, 10, 2, 187, 195]-(positive)
[91, 39, 83, 176, 59, 7, 169, 39, 20, 132, 190, 426, 142, 67, 0, 24, 192, 199]-(negative)
[98, 58, 101, 208, 65, 12, 226, 30, 25, 182, 225, 748, 216, 71, 6, 1, 185, 196]-(positive)
[89, 47, 85, 147, 58, 10, 153, 44, 19, 151, 175, 349, 186, 74, 13, 7, 186, 197]-(negative)
[89, 44, 82, 136, 54, 6, 149, 45, 19, 144, 170, 332, 168, 68, 10, 14, 188, 193]-(positive)
[95, 47, 81, 176, 59, 7, 168, 39, 20, 152, 196, 42